In [1]:
import json
import numpy as np

In [2]:
friend_data = np.genfromtxt('grupee_data/friends.csv', delimiter=',', dtype=int)
friendships = [[] for _ in range(np.max(friend_data[:,0])+1)]

for id in set(friend_data[:,0]):
    friendships[id] = [id, friend_data[friend_data[:, 0] == id, 1]]

with open('grupee_data/preferences.json') as file:
    preferences = json.load(file)

with open('grupee_data/n_concerts.txt') as file:
    n_concerts = file.read().split('\n')[1:-1]
    concerts = np.array([(x.split(':')[0], x.split(':')[1]) for x in n_concerts])
    file.close()

In [3]:
friendship_matrix = np.empty((len(friendships), len(friendships)), dtype=int)

for i, id in enumerate(friendships):
    row = np.zeros(len(friendships))
    if id:
        row[id[1]] = 1
        friendship_matrix[i] = row
    else:
        friendship_matrix[i] = row
        
friendship_matrix = friendship_matrix + friendship_matrix.T - np.diag(friendship_matrix.diagonal())

In [54]:
degreeMat = np.diag(np.sum(friendship_matrix, axis=1))
laplacian = degreeMat - friendship_matrix

degrees = np.empty((len(friendships), 2))

for id in range(len(friendships)):
    degrees[id] = (id, laplacian[id][id])

print(f'12% of {len(friendships)}: {len(friendships) * .12}')
sortedDegrees = sorted(degrees, key=lambda tup: tup[1])
mostInfluential = sortedDegrees[len(friendships)-round(len(friendships) * .12):]
print(mostInfluential)

12% of 8311: 997.3199999999999
[array([1393.,   25.]), array([1527.,   25.]), array([1546.,   25.]), array([1681.,   25.]), array([1688.,   25.]), array([1735.,   25.]), array([1761.,   25.]), array([1789.,   25.]), array([1796.,   25.]), array([1827.,   25.]), array([1857.,   25.]), array([1869.,   25.]), array([1890.,   25.]), array([1908.,   25.]), array([1928.,   25.]), array([1989.,   25.]), array([2191.,   25.]), array([2240.,   25.]), array([2264.,   25.]), array([2271.,   25.]), array([2505.,   25.]), array([2729.,   25.]), array([2754.,   25.]), array([2832.,   25.]), array([2852.,   25.]), array([2855.,   25.]), array([3007.,   25.]), array([3082.,   25.]), array([3344.,   25.]), array([3355.,   25.]), array([3365.,   25.]), array([3397.,   25.]), array([3467.,   25.]), array([3596.,   25.]), array([3753.,   25.]), array([3765.,   25.]), array([3849.,   25.]), array([3864.,   25.]), array([3901.,   25.]), array([3970.,   25.]), array([4381.,   25.]), array([4733.,   25.]), ar